# Aufgabe 12.3: Regressionsmodell für Endgewicht

- **Abgabeformalien:** Fügen sie ein Kapitel in ihrer Dokumentation hinzu, in dem Sie die Ergebnisse der Regression dokumentieren und geben Sie die Datei `reg_<Matrikelnummer1-Matrikelnummer2-Matrikelnummer3>.csv` mit ab
- Erstellen Sie ein Lineares Regressionsmodell zur Vorhersage des Endgewichts anhand aller sinnvollen Daten
- Erstellen Sie für Ihren Report eine Tabelle, welche die genuzten Spalten (X) für die Vorhersage (y) enthält und den MSE-Wert für die jeweiligen Spalten
  
| Genutzte Spalten                            | Modell-Typ | MSE-Wert (Training)  | MSE-Wert (Test) |
|---------------------------------------------|------------|----------------------|-----------------|
| [Vibration_index_blue]                      | Linear     | 0.5                  | 0.6             |
| [Vibration_index_blue, Vibration_index_blue]| Logistic   | 0.7                  | 0.8             |
| Spalte 1, Spalte 2                          | SVM        | 0.9                  | 1.0             |

- Schreiben Sie die Formel für das beste Lineare Regressionsmodell auf in der Form y=mx+b incl. der Parameter
- Machen Sie eine Prognose für das folgende Datenset X.csv mit Ihrem besten Modell
- Als Orientierung kann folgendes Notebook dienen docs/8_Regression_Python.ipynb, welches auch im nächsten Abschnitt vorgestellt wird
- Speichern Sie die Prognose in einer CSV-Datei `reg_<Matrikelnummer1-Matrikelnummer2-Matrikelnummer3>.csv` und dokumentieren Sie Ihre Ergebnisse in der Markdown-Datei

---

## Vorbereiten der Daten
Als erster lesen wir die Daten aus der Datenbank aus und bereiten sie für die Regression vor. Wir nutzen dafür die Pandas-Bibliothek, um die Daten in einen `Dataframe` zu laden und zu verarbeiten.

Dadurch dass die Daten schon gut greinigt im `mqtt_data.json` vorliegen, kann sich viel preprocessing erspart bleiben.

In [ ]:
import json
import pandas as pd
import seaborn as sns

with open("../mqtt_data.json", "r") as f:
    data = json.load(f)

# --- Final Weight DataFrame ---
final_weight_entries = data.get("final_weight", {})
dispencer_red_entries = data.get("dispencer_red", {})
dispencer_green_entries = data.get("dispencer_green", {})
dispencer_blue_entries = data.get("dispencer_blue", {})

df_final_weight = pd.DataFrame([
    {
        "bottle": entry["bottle"],
        "time": pd.to_datetime(entry["time"], unit="s"),
        "final_weight": entry["final_weight"],
        "fill_level_red": None,
        "fill_level_green": None,
        "fill_level_blue": None
    }
    for entry in final_weight_entries.values()
])

dispencer_red_entries = data.get("dispenser_blue",{})
df_dispenser_red = pd.DataFrame([
    {
        "bottle": entry["bottle"],
        "fill_level_red": entry["fill_level_grams"]
    }
    for entry in dispencer_red_entries.values()
])

dispencer_green_entries = data.get("dispenser_green",{})
df_dispenser_green = pd.DataFrame([
    {
        "bottle": entry["bottle"],
        "fill_level_green": entry["fill_level_grams"]
    }
    for entry in dispencer_green_entries.values()
])   

dispencer_blue_entries = data.get("dispenser_blue",{})
df_dispenser_blue = pd.DataFrame([
    {
        "bottle": entry["bottle"]
        "fill_level_blue": entry["fill_level_grams"]
    }
    for entry in dispencer_blue_entries.values()
])

df_regression = final_weight_entries.copy()
df_regression = pd.merge(df_regression, df_dispenser_red, on='bottle', how='left')
df_regression = pd.merge(df_regression, df_dispenser_green, on='bottle', how='left')
df_regression = pd.merge(df_regression, df_dispenser_blue, on='bottle', how='left')

print(df_regression)


TypeError: unhashable type: 'list'

## Trainings- und Testdaten

Nun da die Daten sich in sauber in einem Dataframe befinden, kann das Trainieren des Modells beginnen. Dazu werden die Daten in Trainings- und Testdaten unterteilt. Dies ist wichtig, um das Modell auf neuen, unbekannten Daten zu testen und zu validieren.
Die Unterteilung erfolgt mittels k-Fold Cross-Validation, `sk-learn` bietet dafür schon eine Funktion an.
Dies ermöglicht eine bessere Generalisierung des Modells.

Die zwei Modelle die gewählt wurde sind `LinearRegression` und `SVR` (Support Vector Regression). Diese Modelle sind gut geeignet, um die Beziehung zwischen den Eingangsvariablen und dem Endgewicht zu modellieren.

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

def evaluate_model(model, X, y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    mse_train, mse_test = [], []

    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        mse_train.append(mean_squared_error(y_train, y_train_pred))
        mse_test.append(mean_squared_error(y_test, y_test_pred))

    return np.mean(mse_train), np.mean(mse_test)

# Features & Ziel definieren
X = df_final_weight[['final_weight']]  # z.B. auch mehr Spalten möglich
y = df_final_weight['final_weight']

# Ergebnis-DataFrame vorbereiten
df_predictions = pd.DataFrame(columns=[
    "Genutzte Spalten",
    "Modell-Typ",
    "MSE-Wert (Training)",
    "MSE-Wert (Test)"
])

# Liste der Modelle
models = [
    ("Linear", LinearRegression()),
    ("SVM", SVR())
]

# Evaluation für jedes Modell durchführen und eintragen
for model_name, model in models:
    mse_train, mse_test = evaluate_model(model, X, y, k=9)

    df_predictions.loc[len(df_predictions)] = [
        list(X.columns),  # Genutzte Spalten als Liste
        model_name,
        round(mse_train, 4),
        round(mse_test, 4)
    ]

print(df_predictions)

  Genutzte Spalten Modell-Typ  MSE-Wert (Training)  MSE-Wert (Test)
0   [final_weight]     Linear               0.0000           0.0000
1   [final_weight]        SVM               0.0254           0.0267
